## Imports and declerations

In [133]:
import pickle
import os
import string
import re
import pprint
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import boolean
import pyparsing
from pyparsing import Word, alphas, oneOf, operatorPrecedence, opAssoc
InteractiveShell.ast_node_interactivity = "all"

os.getcwd()

'C:\\Users\\hassa\\Desktop\\Development\\Uni Projects\\Information Retrieval\\A1'

## Raw Vocabulary Storage:

In [2]:

# Break words like Veterans.Before, West.In amendment.Change

def split_words(vocabl):
    new_vocab = set()
    for word in vocabl:
        if re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word) is not None:
            print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split('.')
#             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        elif re.search('^[a-zA-Z]+[?][a-zA-Z]+$',word) is not None:
# #             print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split('?')
# #             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        elif re.search('^[a-zA-Z]+[,][a-zA-Z]+$',word) is not None:
# #             print(re.search('^[a-zA-Z]+[.][a-zA-Z]+$',word))
            w1, w2 = word.split(',')
# #             print(w1)
# #             print(w2)
            new_vocab.add(w1)
            new_vocab.add(w2)
        else:
            new_vocab.add(word)
    return new_vocab

# Remove Punctuation
def remove_punctuation(word):
    return word.translate(word.maketrans('','',string.punctuation))


In [114]:
vocab = set()
doc_contents = []
printable = set(string.printable) 
# Printable characters are
# 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
# !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c


stop_words = set()
with open('Stopword-List.txt', 'r') as stop_word_file:
    lines = stop_word_file.readlines()
    for line in lines:
        stop_words.add(line.split('\n')[0])
    stop_words.remove('')

for file_number in range(0, 56):
    with open(f'data/Trump Speechs/speech_{file_number}.txt', 'r') as file1:
        lines = file1.readlines()
#         print(f'File Number : speech_{file_number}.txt' )
#         print(lines[0])
        for line in lines:
            doc_set = set()
            # split words at . , whitespace ? ! : ;
            for word in re.split('[.\s,?!:;]', line):
                
                
                # Case Folding
                word = word.lower()
                
                # Filter non-ASCII characters
                word = ''.join(filter(lambda x: x in printable, word))
                
                # Remove Punctuations
                word = remove_punctuation(word)
                
                if re.match('\d+[A-Za-z]+',word):
                    word = re.split('\d+',word)[1]
                if re.match('[A-Za-z]+\d+',word):
                    word = re.split('\d+',word)[0]
                
                if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                    continue
                if word in stop_words:
                    continue
                         
               
                    
                vocab.add(word)
                
                doc_set.add(word)
        
        doc_contents.append(doc_set)

In [115]:
print('Total Vocabulary Size ')
print(len(vocab))
print('Total Number of Documents ')
print(len(doc_contents))

Total Vocabulary Size 
7459
Total Number of Documents 
56


# Boolean Model:

In [116]:
# print(sorted(list(vocab)))
# for index,doc in enumerate(doc_contents):
#     print('Vocab size of doc' + str(index))
#     print(len(doc))

vocab_list = sorted(list(vocab))

term_doc_matrix_np = np.zeros((len(vocab), len(doc_contents)))

for word_index, word in enumerate(vocab_list):
    word_row = []
    for doc_index, doc in enumerate(doc_contents):
        if word in doc:
            term_doc_matrix_np[word_index, doc_index] = 1
        else:
            term_doc_matrix_np[word_index, doc_index] = 0
            
print(term_doc_matrix_np)
    

[[1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [89]:

with open('pickled/vocab.p', 'ab') as vocab_file:
    pickle.dump(vocab, vocab_file)

In [90]:
with open('pickled/vocab.p', 'rb') as vocab_file:
    vocabf = pickle.load(vocab_file)

In [117]:
query = input('Enter your query : ')
if query in vocab:
    term_index = vocab_list.index(query)
    term_row = term_doc_matrix_np[term_index]
    print(term_row)
    doc_ids = np.argwhere(term_row == 1)
    print(doc_ids)
else:
    print(f'{query} not present in vocabulary')


Enter your query : running
[1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 6]
 [ 8]
 [ 9]
 [10]
 [11]
 [16]
 [19]
 [20]
 [21]
 [22]
 [24]
 [25]
 [26]
 [27]
 [30]
 [32]
 [33]
 [34]
 [35]
 [36]
 [37]
 [39]
 [40]
 [41]
 [45]
 [46]]


In [66]:
test = '000dollar'
x = re.match('\d+[A-Za-z]+',test)
print(x)
x = re.split('\d+',test)
print(x)
# For matching queries like
# not hammer or pakistan
# (magnum or not hammer) or not (polish and pakistan)
x = re.match('(not)?\s*(\w+|(\((not)?\s*(\w+)\s+(and|or)\s+(not)?\s*(\w+)\)))\s+(or|and)\s+(not)?\s*(\w+|(\((not)?\s*(\w+)\s+(and|or)\s+(not)?\s*(\w+)\)))')



<re.Match object; span=(0, 9), match='000dollar'>
['', 'dollar']


In [99]:
ans = {'0', '1', '10', '11', '12', '16', '17', '18', '19', '2', '20', '21', '22', '24', '25', '26', '27', '28', '3', '30', '32', '33', '34', '35', '36', '37', '39', '4', '40', '41', '44', '45', '46', '47', '5', '50', '51', '52', '53', '6', '8', '9'}
ans2 =set([str(x[0]) for x in doc_ids])

In [108]:
print(len(ans))
print((ans2))
print(ans.difference(ans2))

42
{'19', '11', '35', '41', '2', '6', '40', '30', '36', '1', '0', '34', '16', '22', '8', '9', '4', '45', '37', '39', '26', '46', '21', '32', '25', '33', '3', '24', '27', '10', '20'}
{'52', '44', '12', '53', '51', '50', '5', '17', '18', '47', '28'}


# References :

http://www.pyregex.com/
http://cs231n.github.io/python-numpy-tutorial/
https://www.online-utility.org/text/analyzer.jsp
https://stackoverflow.com/questions/2118261/parse-boolean-arithmetic-including-parentheses-with-regex
https://regex101.com/r/M8z3U4/1

pakistan|afganistan|aid


TypeError: 'set' object is not callable

In [ ]:
inp = "pakistan AND Running"
 re.split('[\s\(\)]', inp)